Есть набор данных о стоимости недвижимости в Великобритании - .csv файл (описание столбцов тут https://www.gov.uk/guidance/about-the-price-paid-data)
нужно написать программу, которая сформирует файл, в котором будет перечислена вся недвижимость, проданная больше 1-го раза.
Программа должна потреблять как можно меньше вычислительных ресурсов. Рассмотреть случаи экономии памяти и процессорного времени.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Анализ с помощью pandas

In [2]:
import pandas as pd

In [3]:
#Датасет достаточно большой и не загружается в память полностью(по крайней мере с 12 gb ram в google colab). 
#Поэтому загружаем только нужные для анализа колонки
#Это id транзакции, тип транзакции(PPD Category Type), статус записи(Record Status), а также тип собственности и ее адрес.
#Будем считать, что собственность полностью характеризуется адресом и типом
#(адрес может меняться, но учесть это только с помощью этого датасета невозможно) 
df0 = pd.read_csv('/content/drive/MyDrive/datasets/pp-complete.csv', header=None, usecols=[0, 4,7, 8,9,10,11,12,13,14,15])

In [4]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26541204 entries, 0 to 26541203
Data columns (total 11 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   0       object
 1   4       object
 2   7       object
 3   8       object
 4   9       object
 5   10      object
 6   11      object
 7   12      object
 8   13      object
 9   14      object
 10  15      object
dtypes: object(11)
memory usage: 2.2+ GB


In [5]:
#Вставляем оригинальные названия колонок
df0.rename(columns={
    0: 'id',
    4: 'Property Type',
    7: 'PAON',
    8: 'SAON',
    9: 'Street', 
    10: 'Locality', 
    11: 'Town/City', 
    12: 'District',
    13: 'County', 
    14: 'PPD Category Type', 
    15: 'Record Status'}, inplace=True) 

In [6]:
df0.head()

,id,Property Type,PAON,SAON,Street,Locality,Town/City,District,County,PPD Category Type,Record Status
0,{F887F88E-7D15-4415-804E-52EAC2F10958},D,31,NaN,ALDRICH DRIVE,WILLEN,MILTON KEYNES,MILTON KEYNES,MILTON KEYNES,A,A
1,{40FD4DF2-5362-407C-92BC-566E2CCE89E9},T,50,NaN,HOWICK PARK,SUNDERLAND,SUNDERLAND,SUNDERLAND,TYNE AND WEAR,A,A
2,{7A99F89E-7D81-4E45-ABD5-566E49A045EA},T,19,NaN,BRICK KILN CLOSE,COGGESHALL,COLCHESTER,BRAINTREE,ESSEX,A,A
3,{28225260-E61C-4E57-8B56-566E5285B1C1},T,37,NaN,RAINSBROOK DRIVE,SHIRLEY,SOLIHULL,SOLIHULL,WEST MIDLANDS,A,A
4,{444D34D7-9BA6-43A7-B695-4F48980E0176},S,59,NaN,MERRY HILL,BRIERLEY HILL,BRIERLEY HILL,DUDLEY,WEST MIDLANDS,A,A


In [7]:
#Пропущенные значения встречаются в адресе собственности. 
#Считаем, что это нормально, например у частного дома нет адреса квартиры(SAON) и т.д.
df0.isna().sum()

id                          0
Property Type               0
PAON                     4194
SAON                 23445666
Street                 416293
Locality              9004011
Town/City                   0
District                    0
County                      0
PPD Category Type           0
Record Status               0
dtype: int64

рассмотрим технические колонки PPD Category Type и Record Status

In [8]:
df0['Record Status'].value_counts()

A    26541204
Name: Record Status, dtype: int64

В Record Status нет значений 'C' и 'D', то есть нет пометок на удаление и изменение данных. Соответственно, не нужно удалять строки. Дополнительно проверим транзакции на уникальность id

In [9]:
df0['id'].is_unique

True

In [10]:
df0['PPD Category Type'].value_counts()

A    25549153
B      992051
Name: PPD Category Type, dtype: int64

In [11]:
#Таким образом можно уменьшить объем занимаемый датасетом,
#удалив ненужные колонки(или в принципе не загружать их изначально)
df0.drop(labels=['id', 'Record Status', 'PPD Category Type'],axis = 1)

,Property Type,PAON,SAON,Street,Locality,Town/City,District,County
0,D,31,NaN,ALDRICH DRIVE,WILLEN,MILTON KEYNES,MILTON KEYNES,MILTON KEYNES
1,T,50,NaN,HOWICK PARK,SUNDERLAND,SUNDERLAND,SUNDERLAND,TYNE AND WEAR
2,T,19,NaN,BRICK KILN CLOSE,COGGESHALL,COLCHESTER,BRAINTREE,ESSEX
3,T,37,NaN,RAINSBROOK DRIVE,SHIRLEY,SOLIHULL,SOLIHULL,WEST MIDLANDS
4,S,59,NaN,MERRY HILL,BRIERLEY HILL,BRIERLEY HILL,DUDLEY,WEST MIDLANDS
...,...,...,...,...,...,...,...,...
26541199,D,40,NaN,FLAT HOLM WALK,SULLY,PENARTH,THE VALE OF GLAMORGAN,THE VALE OF GLAMORGAN
26541200,D,"LINDERIC, 2B",NaN,PANT GLAS,NaN,ST ASAPH,DENBIGHSHIRE,DENBIGHSHIRE
26541201,D,3,NaN,CLOS OAKDALE,GELLIHAF,BLACKWOOD,CAERPHILLY,CAERPHILLY
26541202,D,32,NaN,MELROSE WALK,SULLY,PENARTH,THE VALE OF GLAMORGAN,THE VALE OF GLAMORGAN


PPD Category Type имеет два типа транзакций, но учитывая, что id уникальные, данные корректны.

In [12]:
#теперь сгруппируем транзакции по колонкам, соответствующим адресу и типу собственности
#и посчитаем их количество в группах. Это и будет число продаж данной собственности
df0 = df0.groupby(['Property Type', 'PAON', 'SAON', 'Street', 'Locality',
       'Town/City', 'District', 'County']).size()

In [13]:
#res.index = res.index.to_flat_index()

In [14]:
#res.index.get_level_values(0)

In [15]:
#Находим собственность проданную более 1 раза, и обнуляем иерархический индекс(полученный при группировке)
res2 = df0[df0 > 1].reset_index()

In [16]:
#найдено 359291 объект
res2.rename(columns={0: 'times_sold'}, inplace=True)
res2

,Property Type,PAON,SAON,Street,Locality,Town/City,District,County,times_sold
0,D,1,AGATHA COTTAGE,OXFORD SQUARE,WATCHFIELD,SWINDON,VALE OF WHITE HORSE,OXFORDSHIRE,3
1,D,1,ANTIQUA HOUSE,SNOWBELL ROAD,KINGSNORTH,ASHFORD,ASHFORD,KENT,2
2,D,1,BAILEY COTTAGE,ANDREWS YARD,ASCOTT UNDER WYCHWOOD,CHIPPING NORTON,WEST OXFORDSHIRE,OXFORDSHIRE,3
3,D,1,BASEMENT FLAT,SOUTH TERRACE,BOSTON,BOSTON,BOSTON,LINCOLNSHIRE,2
4,D,1,BEECH COTTAGE,DE BRIANE CLOSE,HAZELBURY BRYAN,STURMINSTER NEWTON,NORTH DORSET,DORSET,4
...,...,...,...,...,...,...,...,...,...
359286,T,YSTRAD,FLAT 4,HIGH STREET,LLANDRINDOD WELLS,LLANDRINDOD WELLS,POWYS,POWYS,2
359287,T,ZEITAL HOUSE,145,BOW COMMON LANE,LONDON,LONDON,TOWER HAMLETS,GREATER LONDON,2
359288,T,ZION COTTAGES,1,HIGH STREET,SOUTHSEA,WREXHAM,WREXHAM,WREXHAM,3
359289,T,ZION COTTAGES,2,RANTERS LANE,GOUDHURST,CRANBROOK,TUNBRIDGE WELLS,KENT,3


In [17]:
res2[res2['times_sold']>10]

,Property Type,PAON,SAON,Street,Locality,Town/City,District,County,times_sold
115287,F,"BIRCHMOOR, 2",FLAT 1-6,ST MICHAELS ROAD,AIGBURTH,LIVERPOOL,LIVERPOOL,MERSEYSIDE,11
138563,F,CHURCHGATE COURT,FLAT 3,CHURCHGATE,BRAMHOPE,LEEDS,LEEDS,WEST YORKSHIRE,11
168836,F,"FRESH, 138",APARTMENT 1008,CHAPEL STREET,SALFORD,SALFORD,SALFORD,GREATER MANCHESTER,14


In [18]:
res2.to_csv('/content/drive/MyDrive/datasets/pd_pred_pandas.csv')

In [19]:
#пометим загруженный датасет на удаление из памяти(хотя мгновенное удаление и не гарантируется)
del [[res2, df0]]

In [20]:
import gc
gc.collect()

100

# Чтобы использовать меньше оперативной памяти и загрузить весь датасет, будем использовать DASK. DASK позволяет хранить датасет на жестком диске и использовать существенно меньше RAM. Ценой этому является меньшая производительность, поскольку данные загружаются с диска. DASK имеет аналогичный pandas интерфейс, поэтому код практически не изменится.

In [21]:
!python -m pip install dask[dataframe] --upgrade

In [22]:
import numpy as np
import pandas as pd

import dask.dataframe as dd
import dask.array as da
import dask.bag as db

In [23]:
df = dd.read_csv('/content/drive/MyDrive/datasets/pp-complete.csv', header=None)

In [24]:
df = df.rename(columns={
    0: 'id',
    1: 'Price',
    2: 'Date of Transfer',
    3: 'Postcode',
    4: 'Property Type',
    5: 'Old/New',
    6: 'Duration',
    7: 'PAON',
    8: 'SAON',
    9: 'Street', 
    10: 'Locality', 
    11: 'Town/City', 
    12: 'District',
    13: 'County', 
    14: 'PPD Category Type', 
    15: 'Record Status'}) 

In [25]:
df.head()

,id,Price,Date of Transfer,Postcode,Property Type,Old/New,Duration,PAON,SAON,Street,Locality,Town/City,District,County,PPD Category Type,Record Status
0,{F887F88E-7D15-4415-804E-52EAC2F10958},70000,1995-07-07 00:00,MK15 9HP,D,N,F,31,NaN,ALDRICH DRIVE,WILLEN,MILTON KEYNES,MILTON KEYNES,MILTON KEYNES,A,A
1,{40FD4DF2-5362-407C-92BC-566E2CCE89E9},44500,1995-02-03 00:00,SR6 0AQ,T,N,F,50,NaN,HOWICK PARK,SUNDERLAND,SUNDERLAND,SUNDERLAND,TYNE AND WEAR,A,A
2,{7A99F89E-7D81-4E45-ABD5-566E49A045EA},56500,1995-01-13 00:00,CO6 1SQ,T,N,F,19,NaN,BRICK KILN CLOSE,COGGESHALL,COLCHESTER,BRAINTREE,ESSEX,A,A
3,{28225260-E61C-4E57-8B56-566E5285B1C1},58000,1995-07-28 00:00,B90 4TG,T,N,F,37,NaN,RAINSBROOK DRIVE,SHIRLEY,SOLIHULL,SOLIHULL,WEST MIDLANDS,A,A
4,{444D34D7-9BA6-43A7-B695-4F48980E0176},51000,1995-06-28 00:00,DY5 1SA,S,N,F,59,NaN,MERRY HILL,BRIERLEY HILL,BRIERLEY HILL,DUDLEY,WEST MIDLANDS,A,A


In [26]:
res = df.groupby(['Property Type', 'PAON', 'SAON', 'Street', 'Locality',
       'Town/City', 'District', 'County']).size().compute()

In [27]:
res2 = res[res > 1].reset_index()

In [28]:
res2.rename(columns={0: 'times_sold'}, inplace=True)
res2

,Property Type,PAON,SAON,Street,Locality,Town/City,District,County,times_sold
0,D,10,APARTMENT 1,ALEXANDRA DRIVE,LIVERPOOL,LIVERPOOL,LIVERPOOL,MERSEYSIDE,2
1,D,10,FLAT C,LEAMINGTON PARK,LONDON,LONDON,EALING,GREATER LONDON,2
2,D,11,FLAT 1,QUEENS ROAD,RICHMOND,RICHMOND,RICHMOND UPON THAMES,GREATER LONDON,3
3,D,18,GROUND FLOOR FLAT,SOUTH TOWN,DARTMOUTH,DARTMOUTH,SOUTH HAMS,DEVON,3
4,D,18,LAND TO THE REAR OF,LAND TO THE REAR OF,ROSETREES,LYMINGTON,NEW FOREST,HAMPSHIRE,2
...,...,...,...,...,...,...,...,...,...
359286,S,MOORHEY,2,LEA LANE,LEA TOWN,PRESTON,PRESTON,LANCASHIRE,2
359287,T,THE BARN,2,DISHFORTH VILLAGE,DISHFORTH,THIRSK,HARROGATE,NORTH YORKSHIRE,2
359288,F,BLOCK 3 THE RISINGS,APARTMENT 35,BRIDGE ROAD,OLD ST MELLONS,CARDIFF,CARDIFF,CARDIFF,2
359289,O,DYFFRYN COURT,UNIT 7,MOORHEN CLOSE,SWANSEA VALE,SWANSEA,SWANSEA,SWANSEA,2


In [29]:
#Получен результат аналогичный Pandas(по количеству строк и по трем наиболее продаваемым собственностям)
res2[res2['times_sold']>10]

,Property Type,PAON,SAON,Street,Locality,Town/City,District,County,times_sold
75441,F,"BIRCHMOOR, 2",FLAT 1-6,ST MICHAELS ROAD,AIGBURTH,LIVERPOOL,LIVERPOOL,MERSEYSIDE,11
144521,F,CHURCHGATE COURT,FLAT 3,CHURCHGATE,BRAMHOPE,LEEDS,LEEDS,WEST YORKSHIRE,11
320204,F,"FRESH, 138",APARTMENT 1008,CHAPEL STREET,SALFORD,SALFORD,SALFORD,GREATER MANCHESTER,14


In [30]:
res2.to_csv('/content/drive/MyDrive/datasets/pd_pred_dask.csv')